In [1]:
'''
!pip install requests
!pip install pandas
!pip install selenium
!pip install webdriver_manager
'''

'\n!pip install requests\n!pip install pandas\n!pip install selenium\n!pip install webdriver_manager\n'

In [2]:
import os
import time
import sqlite3
import requests
import pandas as pd
import concurrent.futures
from threading import Thread
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
url_full = 'https://tiki.vn/api/v2/reviews?limit=20&include=comments,contribute_info,attribute_vote_summary&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&product_id='
url = 'https://tiki.vn/api/v2/reviews?limit=5&include=comments,contribute_info,attribute_vote_summary&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&spid=10005246&product_id=10005245&seller_id=1'
header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

In [4]:
def check_array(json, name):
      arr = json.get(name)
  if len(arr):
    x = ''
    for i in sorted(arr):
      x = x + str(i) + ' \n'
    return x
  else:
    return ''

In [5]:
def comment_parser(json):
    d = dict()
    d['id'] = json.get('id')
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['status']  = json.get('status')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['comment_count']  = json.get('comment_count')
    d['rating'] = json.get('rating')
    # d['created_by'] = json.get('created_by')
    d['suggestions'] = check_array(json, 'suggestions')
    # d['attributes'] = check_array(json, 'attributes')
    d['spid'] = json.get('spid')
    d['is_photo'] = json.get('is_photo')
    d['seller'] = json.get('seller').get('name')
    d['product_id'] = json.get('product_id')
    d['timeline'] = json.get('timeline').get('content')
    # d['comments'] = json.get('comments')
    d['purchased'] = json.get('created_by').get('purchased')
    return d

In [6]:
def crawl_one_page(data, url, header):
  request = requests.get(url, headers = header)
  for comment in request.json().get('data'):
    data.append(comment_parser(comment))

In [7]:
def crawl_full_page(url, header):
  threads = []
  request = requests.get(url, headers = header)
  if request.status_code != 200:
    print(request.status_code)
  number_of_page = request.json()['paging']['last_page']
  df = []
  for i in range(0, number_of_page):
    x = url.replace('page=1', 'page=' + str(i + 1))
    t = Thread(target = crawl_one_page, args = (df, x, header))
    threads.append(t)
  [ t.start() for t in threads ]
  [ t.join() for t in threads ]
  return pd.DataFrame(df)

In [25]:
api = 'https://tiki.vn/api/v2/products?limit=100&include=advertisement&aggregations=2&trackity_id=01cb163f-90a0-eddb-c5bb-1cd30b00e5c3&q=%C4%91%E1%BA%AFc+nh%C3%A2n+t%C3%A2m&rating=1'


In [26]:
request = requests.get(api, headers = header)
if request.status_code != 200:
    print(request.status_code)
js = request.json().get('data')
crawl_id = pd.DataFrame(js)
product_id = crawl_id['id'].unique()
len(product_id)

113

In [27]:
df = crawl_full_page(url_full + str(product_id[0]), header)
df.to_csv('dataTiki.csv',encoding = 'utf-8-sig', index=False)
for i in range(1,len(product_id)):
    df = crawl_full_page(url_full + str(product_id[i]), header)
    df.to_csv('dataTiki.csv',encoding = 'utf-8-sig', mode = 'a', index=False,header = False)


Exception in thread Thread-2323:
Traceback (most recent call last):
  File "c:\Users\ADMIN\.conda\envs\tf_gpu\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\ADMIN\.conda\envs\tf_gpu\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19216\1963684776.py", line 4, in crawl_one_page
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19216\2795354736.py", line 18, in comment_parser
AttributeError: 'NoneType' object has no attribute 'get'
Exception in thread Thread-2320:
Traceback (most recent call last):
  File "c:\Users\ADMIN\.conda\envs\tf_gpu\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\ADMIN\.conda\envs\tf_gpu\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19216\1963684776.py", line 4, in crawl_one_page
  File "C:\Users\ADMIN\AppData\Local\Temp\ipyk

In [28]:
data = pd.read_csv('./dataTiki.csv')
data.shape

(37540, 15)

In [32]:
data.shape

(37540, 15)

In [45]:
data_model = data.loc[(data.content.isnull() == False) & (data.content.duplicated() == False),["content","rating"]]
data_model.rating.value_counts().sum()

11070

In [49]:
data_model.at[0,'content']

'Mình k ở nhà, nhờ ngừoi nhà nhận hộ thì hộp như thế này đây, mở ra được luôn. Một bên bìa hộp thì rách đôi (có video nhưng ở đây k có mục up vid). Sách thì có vết như kia. Nhìn ảnh thì cũng bình thường nhưng thực tế nhìn ở bên ngoài và bản thân lại là người đi mua sách mới mà bị như vậy rất là khó chấp nhận. Mong Tiki xem xét lại khâu đóng gói hàng và nhân viên giao hàng.'